In [1]:
# 批量归一化：对数据做标准化处理．

### 背景描述：
# 对简单的浅层网络来说，经过标准化处理后的输入数据能使各个特征分布相近，
# 往往容易训练出有效的模型．但是对于深层的网络结构，由于每层的参数一直在更新，
# 尤其是靠近输出层的信息，计算数值可能会出现剧烈的不稳定性．

### 目的：
# 让整个神经网络在各层的输出数值更稳定．

### 方法：
# 除了对输入ｉnput进行标准归一化，还在训练过程中对批量信息进行归一化．具体为：
# 计算小批量的均值和标准差，让批量信息均值为零，标准差为１．
# 具体又可以细分为对fc做bn，　和对conv做bn.

### 1. 对FC做归一化

FC和CONV的不同点在于，可以简单理解两者的维度不一致．前者只有两维，后者是四维．

假设一个卷积网络的模块由conv,bn,act组成，用函数表示即：

$$\phi \left( \text{BN} \left( \boldsymbol{x}\right) \right)$$

$\text{BN}$的包含$m$个样本的输入$\boldsymbol{x} \in \mathbb{R}^{m \times d}$由仿射变换得到

$$ \boldsymbol{x} = \boldsymbol{W} \boldsymbol{u} + \boldsymbol{b}$$

将输入$\boldsymbol{x}$拆分成$m$个样本，或者说，包含了$m$个样本的小批量其包含的样本分别是：

$$ \mathcal{B} = \{ \boldsymbol{x}^{(1)},\boldsymbol{x}^{(2)},\cdots, \boldsymbol{x}^{(m)} \},\boldsymbol{x}^{(i)} \in \mathbb{R}^d $$ 那么有： 

$$\boldsymbol{y}^{(i)} = \text{BN}(\boldsymbol{x}^{(i)})，\boldsymbol{y}^{(i)} \in \mathbb{R}^d　$$

求小批量$\mathcal{B}$的均值$ \boldsymbol{\mu}_{\mathcal{B}}$和方差$ \boldsymbol \sigma _{\mathcal{B}}^{2} $

$$ \boldsymbol{\mu}_{\mathcal{B}} \leftarrow \frac{1}{m} \Sigma_{i=1}^{m}{\boldsymbol{x}^{(i)}} $$

$$ \sigma _{\mathcal{B}}^{2} \leftarrow \frac{1}{m} \Sigma_{i=1}^{m}{\left( \boldsymbol{x}^{(i)} -  \boldsymbol{\mu}_{\mathcal{B}}\right)^2}$$

对输入的$\boldsymbol{x}^{(i)}$进行标准化运算，得到

$$ \hat{\boldsymbol{x}}^{(i)} \leftarrow \frac{\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_{\mathcal{B}}}{\sqrt{\boldsymbol \sigma _{\mathcal{B}}^{2} + \epsilon}}$$


随后，引入两个可学习的参数，拉伸参数$ \boldsymbol{\gamma} $ 和偏移参数$\boldsymbol{\beta}$．(它们的shape与$\hat{\boldsymbol{x}}^{(i)}$一致)

$$ \boldsymbol{y}^{(i)} \leftarrow \boldsymbol{\gamma} \odot \hat{\boldsymbol{x}}^{(i)} + \boldsymbol{\beta}$$

==　问题：引入后两个可学习的参数$ \boldsymbol{\gamma} $和$\boldsymbol{\beta}$的意义是什么？==

给予了模型灵活性．如果BN存在的意义不大，那么系统可以灵活地学习参数（让$\boldsymbol{\gamma}=\sqrt{\boldsymbol \sigma _{\mathcal{B}}^{2} + \epsilon}, \ \ \boldsymbol{\beta}=\boldsymbol{\mu}_{\mathcal{B}}$）．使得该过程"可逆"．


### 2. 对卷积层做归一化

如图：

![](./5.10_bn_conv2.png)

结合代码这个过程应该会更加清晰：

In [ ]:
# 均值（没有dim=1）
mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
# 方差
var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)

# 训练模式下用当前的均值和方差做标准化
X_hat = (X - mean) / torch.sqrt(var + eps)
# 拉伸和偏移
Y = gamma * X_hat + beta

### 3. 测试过程的归一化

在进行批量图像预测时，我们不希望在预测时，对每个批量重新计算mean和std．一旦批量大小改变，mean和std也会随着改变，输出的预测值就没有一个确定值．

常用方法：通过移动平均来估算整个训练数据集的mean和std, 在预测阶段使用训练集得到的mean和std来估计输出．**（和dropout的做法类似）**

### 4. 从零开始

In [1]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append('../d2lzh/')
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batch_norm(is_training, X, gamma, beta, 
               moving_mean, moving_var, eps, momentum):
    # 测试模式
    if not is_training:
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    # 训练模式
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:   # 全连接层 
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:                   # Conv层
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
                
        # 训练模式下，进行标准化运算
        X_hat = (X - mean) / torch.sqrt(var + eps)
        
        # 更新移动平均的mean和ｖar (更新它，用于测试)
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var  = momentum * moving_var + (1.0 - momentum) * var
        
    # 计算出x_hat后，对其进行拉伸＋偏移
    Y = gamma * X_hat + beta
    return Y, moving_mean, moving_var

In [2]:
### BatchNorm层

class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features)
        elif num_dims == 4:
            shape = (1, num_features, 1, 1)
            
        # 参与求梯度的＂拉伸参数＂和＂偏移参数＂，分别初始化
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        
        # 不参与求梯度的moving_var和moving_mean,初始化为0
        self.moving_mean = torch.zeros(shape)
        self.moving_var  = torch.zeros(shape)
        
    def forward(self, X):
        # 使moving_mean和moving_var的存放位置与输入Ｘ的位置一致
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var  = self.moving_var.to(X.device)
            
        # BatchNorm如何判断当前是训练/测试．默认是训练，当调用train.eval()后为测试
        # 训练时：更新moving_mean/var变量+输出信息；测试时：应用moving信息＋输出
        Y, self.moving_mean, self.moving_var = batch_norm(
                self.training, X, self.gamma, self.beta,  
               self.moving_mean, self.moving_var, eps=1e-5, momentum=0.9)
        
        
        return Y

In [3]:
### LeNet使用ＢＮ

net = nn.Sequential(
        nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
        BatchNorm(6, num_dims=4),
        nn.Sigmoid(),
        nn.MaxPool2d(2, 2), # kernel_size, stride
    
        nn.Conv2d(6, 16, 5),
        BatchNorm(16, num_dims=4), 
        nn.Sigmoid(),
        nn.MaxPool2d(2, 2),
    
        d2l.FlattenLayer(),
        nn.Linear(16*4*4, 120),
        BatchNorm(120, num_dims=2),
        nn.Sigmoid(),
    
        nn.Linear(120, 84),
        BatchNorm(84, num_dims=2),
        nn.Sigmoid(),
    
        nn.Linear(84, 10)
)

In [4]:
### 模型训练

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

lr, num_epochs = 0.001, 20
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.9930, train_acc 0.783, test acc 0.840, time 3.3 sec
epoch 2, loss 0.4543, train_acc 0.866, test acc 0.857, time 3.2 sec
epoch 3, loss 0.3607, train_acc 0.882, test acc 0.887, time 3.2 sec
epoch 4, loss 0.3233, train_acc 0.891, test acc 0.872, time 3.3 sec
epoch 5, loss 0.3030, train_acc 0.895, test acc 0.884, time 3.5 sec
epoch 6, loss 0.2858, train_acc 0.901, test acc 0.893, time 3.3 sec
epoch 7, loss 0.2724, train_acc 0.904, test acc 0.874, time 3.4 sec
epoch 8, loss 0.2641, train_acc 0.907, test acc 0.884, time 3.4 sec
epoch 9, loss 0.2543, train_acc 0.909, test acc 0.901, time 3.3 sec
epoch 10, loss 0.2465, train_acc 0.912, test acc 0.890, time 3.4 sec
epoch 11, loss 0.2405, train_acc 0.915, test acc 0.908, time 3.4 sec
epoch 12, loss 0.2316, train_acc 0.917, test acc 0.874, time 3.3 sec
epoch 13, loss 0.2240, train_acc 0.919, test acc 0.909, time 3.2 sec
epoch 14, loss 0.2189, train_acc 0.921, test acc 0.923, time 3.3 sec
epoch 15, loss 0.2105, tr

In [5]:
### 查看参数gamma和beta
net[1].gamma.view((-1,)), net[1].beta.view((-1,))

(tensor([1.1197, 1.0583, 1.1344, 1.3078, 1.1341, 1.4962], device='cuda:0',
        grad_fn=<ViewBackward>),
 tensor([ 0.0248,  0.3418, -1.1655,  0.1983,  0.2092,  0.1473], device='cuda:0',
        grad_fn=<ViewBackward>))

### 5. Pytorch实现

In [10]:
### LeNet使用ＢＮ

net = nn.Sequential(
        nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
        nn.BatchNorm2d(6), # num_features
        nn.Sigmoid(),
        nn.MaxPool2d(2, 2), # kernel_size, stride
    
        nn.Conv2d(6, 16, 5),
        nn.BatchNorm2d(16), 
        nn.Sigmoid(),
        nn.MaxPool2d(2, 2),
    
        d2l.FlattenLayer(),
        nn.Linear(16*4*4, 120),
        nn.BatchNorm1d(120),
        nn.Sigmoid(),
    
        nn.Linear(120, 84),
        nn.BatchNorm1d(84),
        nn.Sigmoid(),
    
        nn.Linear(84, 10)
)

In [6]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

lr, num_epochs = 0.001, 20
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.1804, train_acc 0.935, test acc 0.933, time 3.4 sec
epoch 2, loss 0.1725, train_acc 0.938, test acc 0.925, time 3.3 sec
epoch 3, loss 0.1661, train_acc 0.939, test acc 0.914, time 3.3 sec
epoch 4, loss 0.1634, train_acc 0.940, test acc 0.946, time 3.2 sec
epoch 5, loss 0.1585, train_acc 0.943, test acc 0.910, time 3.4 sec
epoch 6, loss 0.1547, train_acc 0.944, test acc 0.924, time 3.6 sec
epoch 7, loss 0.1495, train_acc 0.946, test acc 0.924, time 3.4 sec
epoch 8, loss 0.1448, train_acc 0.949, test acc 0.929, time 3.5 sec
epoch 9, loss 0.1375, train_acc 0.951, test acc 0.935, time 3.8 sec
epoch 10, loss 0.1352, train_acc 0.951, test acc 0.917, time 3.2 sec
epoch 11, loss 0.1302, train_acc 0.953, test acc 0.944, time 3.4 sec
epoch 12, loss 0.1272, train_acc 0.954, test acc 0.943, time 3.4 sec
epoch 13, loss 0.1218, train_acc 0.956, test acc 0.938, time 3.4 sec
epoch 14, loss 0.1185, train_acc 0.958, test acc 0.912, time 3.3 sec
epoch 15, loss 0.1147, tr